Taller #3 Info II - Isabela Muñoz, Lauren Cordero, Paula Garcia

In [1]:
pip install pydicom 

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\imv01\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
import os
import numpy as np
import pydicom
import cv2
import matplotlib.pyplot as plt

In [8]:
pacientes = {}
archivos = {}

class Paciente:
    def __init__(self, identificacion, edad, nifti):
        self.__id = identificacion
        self.__edad = edad
        self.__nifti = nifti

    def info (self):
        return f"Información del paciente: ID: {self.__id}, Edad: {self.__edad}, Imagen asociada: {self.__nifti}"
    
    def leer_archivos(carpeta):
        archivos_dicom = [f for f in os.listdir(carpeta) if f.lower().endswith(".dcm")]
        ruta = os.path.join(carpeta, archivos_dicom[0])
        ds = pydicom.dcmread(ruta)
        id = ds.PatientID
        edad = ds.PatientAge
        dimension = [pydicom.dcmread(os.path.join(carpeta, file)).pixel_array for file in archivos_dicom]
        dimension_array = np.stack(dimension, axis=-1)
        paciente = Paciente(id, edad, dimension_array)

        
        archivos[id] = archivos_dicom
        return paciente
     

    def rotacion (imagen, angulo, salida="rotada.png"):
        if angulo not in [90, 180, 270]:
            raise ValueError("El ángulo debe ser 90, 180 o 270 grados.")
        if angulo == 90:
            im_rotada = cv2.rotate(imagen, cv2.ROTATE_90_COUNTERCLOCKWISE)
        elif angulo == 180:
            im_rotada = cv2.rotate(imagen, cv2.ROTATE_180)
        elif angulo == 270:
            im_rotada = cv2.rotate(imagen, cv2.ROTATE_270_CLOCKWISE)
        else:
            print("El ángulo no está permitido")

        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(imagen, cmap='gray')
        plt.title("Imagen Original")
        plt.subplot(1, 2, 2)
        plt.imshow(im_rotada, cmap='gray')
        plt.title(f"Imagen Rotada {angulo}°")
        plt.show()
        cv2.imwrite(salida, im_rotada)
        return im_rotada

    def binarizacion_transformacion(imagen, umbral, kernel, salida="binarizada.png"):
        retval, im_binarizada = cv2.threshold(imagen, umbral, 255, cv2.THRESH_BINARY)
        print("Umbral utilizado:", retval)
        size_kernel = np.ones((kernel, kernel), np.uint8)
        im_transformada = cv2.morphologyEx(im_binarizada, cv2.MORPH_CLOSE, size_kernel)
        texto = f"Imagen binarizada\nUmbral: {umbral}\nKernel: {kernel}x{kernel}"
        cv2.putText(im_transformada, texto, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        plt.imshow(im_transformada, cmap='gray')
        plt.title("Imagen Transformada con Texto") 
        plt.show()
        cv2.imwrite(salida, im_transformada)
        return im_transformada


In [9]:
def mostrar_menu():
    print(f'''
          1. Ingresar Paciente (cargar archivos DICOM)
          2. Ingresar imágenes JPG o PNG
          3. Rotar imagen de archivo DICOM
          4. Transformar imagen JPG o PNG (binarización y morfología)
          5. Salir''')


def opcion_ingresar_paciente():
    carpeta = input("Ingrese la ruta de la carpeta con los archivos DICOM: ")
    try:
        paciente = leer_archivos(carpeta)
        pacientes[paciente.get_id()] = paciente
        print(f"Paciente agregado: {paciente.info()}")
    except Exception as e:
        print(f"Error al cargar el paciente: {e}")

def opcion_ingresar_imagen():
    ruta_imagen = input("Ingrese la ruta de la imagen JPG o PNG: ")
    try:
        nombre_archivo = os.path.basename(ruta_imagen)
        imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
        if imagen is None:
            raise ValueError("No se pudo cargar la imagen. Verifique el formato.")
        archivos[nombre_archivo] = imagen
        print(f"Imagen {nombre_archivo} almacenada correctamente.")
    except Exception as e:
        print(f"Error al cargar la imagen: {e}")

def opcion_rotar_imagen():
    clave = input("Ingrese la clave del archivo DICOM para rotar su imagen: ")
    angulo = int(input("Ingrese el ángulo de rotación (90, 180, 270): "))
    try:
        if clave in archivos:
            imagen = archivos[clave]
            imagen_rotada = rotacion(imagen, angulo)
            archivos[f"{clave}_rotada"] = imagen_rotada
            print(f"Imagen rotada guardada como {clave}_rotada.")
        else:
            print("Clave no encontrada en los archivos.")
    except Exception as e:
        print(f"Error al rotar la imagen: {e}")

def opcion_transformar_imagen():
    clave = input("Ingrese la clave de la imagen JPG o PNG para transformar: ")
    umbral = int(input("Ingrese el umbral para la binarización (0-255): "))
    kernel = int(input("Ingrese el tamaño del kernel para la transformación: "))
    try:
        if clave in archivos:
            imagen = archivos[clave]
            imagen_transformada = binarizacion_transformacion(imagen, umbral, kernel)
            archivos[f"{clave}_transformada"] = imagen_transformada
            print(f"Imagen transformada guardada como {clave}_transformada.")
        else:
            print("Clave no encontrada en los archivos.")
    except Exception as e:
        print(f"Error al transformar la imagen: {e}")

def main():
    while True:
        mostrar_menu()
        m = input("Seleccione una opción: ")
        if m == "1":
            opcion_ingresar_paciente()
        elif m == "2":
            opcion_ingresar_imagen()
        elif m == "3":
            opcion_rotar_imagen()
        elif m == "4":
            opcion_transformar_imagen()
        elif m == "5":
            print("Saliendo del programa.")
            break
        else:
            print("Opción no válida. Intente de nuevo.")

if __name__ == "__main__":
    main()




          1. Ingresar Paciente (cargar archivos DICOM)
          2. Ingresar imágenes JPG o PNG
          3. Rotar imagen de archivo DICOM
          4. Transformar imagen JPG o PNG (binarización y morfología)
          5. Salir


Error al cargar el paciente: name 'leer_archivos' is not defined

          1. Ingresar Paciente (cargar archivos DICOM)
          2. Ingresar imágenes JPG o PNG
          3. Rotar imagen de archivo DICOM
          4. Transformar imagen JPG o PNG (binarización y morfología)
          5. Salir
Saliendo del programa.
